In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import sqlite3
import pandas as pd
import json
import glob
import os.path
from os import listdir,getcwd

##### FAST : Features from Acceletated Segment Test. 
###### Used for Corner Detection. FAST is only a keypoint detector. To extract features, we will need a separate instance of a DescriptorExtractor, like BRISK, SURF,SIFT,ORB etc..  

https://docs.opencv.org/3.4/df/d74/classcv_1_1FastFeatureDetector.html#details

In [2]:
#Initiate FAST object with default values
fast = cv2.FastFeatureDetector_create()

### Keypoints: 
#### Key points are point of interest in an image,that helps to find interesting regions or spatial areas in an image. This is important because no matter how the image changes or transforms, we can able to find same keypoints between the modified and the original images.
#### Parameters in a keypoint:
##### pt	- x & y coordinates of the keypoint
##### size -	keypoint diameter
##### angle	- keypoint orientation
##### response	- keypoint detector response on the keypoint (that is, strength of the keypoint)
##### octave -pyramid octave in which the keypoint has been detected
##### class_id - object id

https://docs.opencv.org/3.4/d5/d51/group__features2d__main.html#ga816d870cbdca71c6790c71bdf17df099

In [3]:
def extractFeatures(path):
    imageX = cv2.imread(path, cv2.IMREAD_UNCHANGED)

#convert image to RGB,because Opencv reads as BGR
    image1 = cv2.cvtColor(imageX, cv2.COLOR_RGBA2BGRA)
#Convert image into gray image
    img = cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)
  
    data = {} #Dictionary to store keypoints
    
    try:
        kp = fast.detect(img,None) # To find keypoints
        total=len(kp) # No. of Keypoints
        print("No of keypoint:",total)
        count = 0
        for i in range(len(kp)):
            for k in kp:
                x=k.pt[0]
                y=k.pt[1] 
                s=k.size
                r=k.response
                o=k.octave
                a=k.angle
                c=k.class_id
        
                data['KeyPoint_%d'%count] = []  
                data['KeyPoint_%d'%count].append({'x': x})
                data['KeyPoint_%d'%count].append({'y': y})
                data['KeyPoint_%d'%count].append({'size':s})
                data['KeyPoint_%d'%count].append({'response': r})
                data['KeyPoint_%d'%count].append({'octave': o})
                data['KeyPoint_%d'%count].append({'angle': a})
                data['KeyPoint_%d'%count].append({'class_id': c})
                count+=1
        Keypoint= json.dumps(data) # To write these dict entries of keypoints into database, convert into JSON
    except cv2.error as e:
        print("Error:",e)
        return None
        
    return (Keypoint, total)

In [4]:
#path= r"C:\Users\kalai\RubikCube.jpg"
#extractFeatures(path)


####  Create DATABASE and TABLE

In [5]:
try:
    conn= sqlite3.connect('ImageFeatures.db')
    c= conn.cursor()
    print("Database created successfully")
except sqlite3.Error as e:
    print("Error:",e)

Database created successfully


In [6]:
try:
    conn = sqlite3.connect('ImageFeatures.db')
    query= '''CREATE TABLE IF NOT EXISTS ImgFeature(Name TEXT PRIMARY KEY,
                                                    Size INTEGER,
                                                    Format TEXT, 
                                                    Channels TEXT,
                                                    KeyPoints JSON,
                                                    Total_Keypoints INTEGER)''';
    c=conn.cursor()
    print("Successfully connected to the database")
    c.execute(query)
    conn.commit()
    print("Table created")
    c.close()
except sqlite3.Error as e:
    print(e)


Successfully connected to the database
Table created


#### Read the image unchanged. Get image details.
###### img.shape: Returns tuple of (height,width,No. of channels). If the value 3, then image has 3 color channels(BGR); If value is 4, then image has 4 channels(B,G,R,A[Transparency channel]) ; If value is 2, then image has zero channels(gray image).


In [7]:
def imageinfo(path):
    
        img= cv2.imread(path,cv2.IMREAD_UNCHANGED)
        size = format(img.size)
        channel=(img.shape[-1]) 
        base=os.path.basename(path)
        filename,ext=os.path.splitext(base) 
               
        return (filename,ext,size,channel)
              

####  INSERT values into the Database

In [8]:
def insertinfo(path):
    try: 
        conn=sqlite3.connect('ImageFeatures.db')
        c=conn.cursor()
        print("Connection successful")
         
        query1='''INSERT INTO ImgFeature(Name,Size,Format,Channels,Keypoints,Total_Keypoints)
                                         VALUES(?,?,?,?,?,?);'''
        result=extractFeatures(path)
        KP= result[0]
        tot= result[1] 
        
        info = imageinfo(path)
        filename= info[0]   
        ext= info[1] 
        size= info[2] 
        channels= info[3] 
        
        data= (filename,size,ext,channels,KP,tot)
        c.execute(query1,data)
        conn.commit()
        print("Image information updated")
        c.close()
        
    except sqlite3.Error as e:
        print("Failed to insert values:", e)
        
    finally:
        if conn:
            conn.close()
            print("Conn closed")
  
    

In [9]:
#To insert one image information 
path =r"C:\Users\kalai\RubikCube.jpg"
insertinfo(path)

Connection successful
No of keypoint: 566
Image information updated
Conn closed


In [10]:
# To load and insert multiple image file info into the database
path =r"C:/Users/kalai/myimages/*.*"    # 5 image files in this path
files= glob.glob(path)

for file in files :
    insertinfo(file)
  

Connection successful
No of keypoint: 1406
Image information updated
Conn closed
Connection successful
No of keypoint: 1020
Image information updated
Conn closed
Connection successful
No of keypoint: 1053
Image information updated
Conn closed
